In [1]:
import numpy as np
import numba as nb
from numba import jit

In [2]:
P=500;        # number of samples to generate from target

T=50;       # length of time series
N=500;      # number of candidates at each time step


# signal and observations
d = 1
xtrue=np.zeros((T, d))
y=np.zeros((T, d))
alpha=0.95   # parameters from Biometrika 2011
beta=0.7
sv=0.3

In [3]:
ss=sv/np.sqrt(1-alpha**2) # starting sigma

In [4]:
xtrue=np.zeros((T, d));
xtrue[0,:]=ss*np.random.randn(d)

In [5]:
y[0]=beta*np.exp(xtrue[0,0]/2)*np.random.randn(d);

In [6]:
for t in np.arange(1,T):
    xtrue[t]=alpha*xtrue[t-1]+sv*np.random.randn(d)
    y[t]=beta*np.exp(xtrue[t]/2)*np.random.randn(d)

In [7]:
wbar        = np.zeros(T)
wbar[0]   = 1/ss
wbar[1:T] = np.ones(T-1)/sv

In [8]:
logZ=0        # estimator of log-normalizing constant
logZbar=0      # bound on the log-normalizing constant

xcand=np.zeros((T,d)) # candidate associated to this proposed path
xacc=np.zeros((P,T))  # exact samples stored


ntrials=0.              # number of trials
pacc=0.                  # estimated acceptance proba
paccave=0.              # estimated by averaging 
paccasq=0.              # store sum square



In [9]:
def random_grid(beta, N, T, y):
    x=np.zeros((T,N,d))
    for t in np.arange(T):
        x[t]=(np.log(y[t]**2)-np.log(beta**2))*np.ones((N,d))-np.log(np.random.randn(N,d)**2)
    return x

In [10]:
def bound_initw(winit, wbar, icand):
    winit[icand[0]] = wbar[0]
    return winit

def bound_w(w, wbar, icand, t):
    w[icand[t],:] = wbar[t]
    w[:, icand[t]] = wbar[t]
    return w

In [11]:

def compute_squared_distances(x, y):
    x2 = np.expand_dims(np.sum(x**2, axis=1),-1)
    y2 = np.expand_dims(np.sum(y**2, axis=1),-1)
    dists = -2 * np.dot(x, y.T) + y2.T + x2
    return dists

In [12]:

def step(x, alpha, sv, predlike, filter_state, llike, t):
    x1=x[t]
    x2=alpha*x[t-1]  
    dists = compute_squared_distances(x1,x2)
    
    logw = dists / (2.*sv**2)
    logwmin=np.min(logw)

    w = np.exp(-logwmin)*np.exp(-logw+logwmin)/sv

    filter_state[t]=w.dot(filter_state[t-1])
    predlike[t]=np.sum(filter_state[t])
    filter_state[t]=filter_state[t]/predlike[t]
    llike=llike+np.log(predlike[t])
    return llike, predlike, filter_state

In [13]:

def forwardHMM(x, alpha, sv):
    predlike=np.zeros(T)  
    filter_state=np.zeros((T,N)) 
    
    #init
    winit = (np.exp(-x[0,:,0]**2./(2*ss**2))/ss)
    filter_state[0] = winit
    predlike[0] = np.sum(filter_state[0])
    llike = np.log(predlike[0])
    filter_state[0] = filter_state[0]/predlike[0]
    
    llike=0. 
    for t in range(1,T):
        llike, predlike, filter_state = step(x, alpha, sv, predlike, filter_state, llike, t)
    return llike, predlike, filter_state

In [14]:

def bound_step(x, alpha, sv, predlike, filter_state, llike, wbar, icand, t):
    x1=x[t]
    x2=alpha*x[t-1]  
    dists = compute_squared_distances(x1,x2)
    
    logw = dists / (2.*sv**2)
    logwmin=np.min(logw)

    w = np.exp(-logwmin)*np.exp(-logw+logwmin)/sv
    w = bound_w(w, wbar, icand, t)
    
    filter_state[t]=w.dot(filter_state[t-1])
    predlike[t]=np.sum(filter_state[t])
    filter_state[t]=filter_state[t]/predlike[t]
    llike=llike+np.log(predlike[t])
    return llike, predlike, filter_state

In [15]:

def forwardHMMbound(x, alpha, sv, icand, wbar):
    predlike=np.zeros(T)  
    filter_state=np.zeros((T,N)) 
    
    #init
    winit = (np.exp(-x[0,:,0]**2./(2*ss**2))/ss)
    winit = bound_initw(winit, wbar, icand)

    
    filter_state[0] = winit
    predlike[0] = np.sum(filter_state[0])
    llike = np.log(predlike[0])
    filter_state[0] = filter_state[0]/predlike[0]
    
    llike=0. 
    
    for t in range(1,T):
        llike, predlike, filter_state = bound_step(x, alpha, sv, predlike, filter_state, llike, wbar, icand, t)
    return llike, predlike, filter_state

In [16]:
def backwardsampling(x, alpha, sv, filter_state):
    
    icand = np.zeros(T, int)
    backfilter = np.zeros(N)
    transition = np.zeros(N)
    
    icand[T-1] = np.random.choice(N,size=1, replace=True, p=filter_state[-1])
    
    for t in np.arange(0, T-1)[::-1]:
        transition = np.exp(-(x[t+1,icand[t+1],:])-alpha*x[t]**2/(2*sv**2))/sv
        backfilter = filter_state[t]*transition.squeeze()
        backfilter = backfilter/np.sum(backfilter) 
        icand[t]   = np.random.choice(N, size=1, replace= True, p=backfilter)
    return icand

In [17]:
n_samples = P
n_acc_samples = 0
acccepted_x = []
cand_x = []
n_trial = 0
while n_acc_samples < n_samples:
    n_trial += 1
    x = random_grid(beta, N, T, y)
    logZ, _, filter_state = forwardHMM(x, alpha, sv)
    icand = backwardsampling(x, alpha, sv, filter_state)
    logZbar, _, _ = forwardHMMbound(x, alpha, sv, icand, wbar)
    pacc= np.exp(logZ-logZbar)
    
    for t in range(T):
        xcand[t] = x[t,icand[t],:]
    
    cand_x.append(xcand.copy())
    u = np.random.random()
    if u < pacc:
        xacc = xcand.copy()
        n_acc_samples += 1
        acccepted_x.append(xacc)
    if n_trial % 100 == 0:
        print(n_acc_samples)
        print(n_acc_samples/n_trial)
    

KeyboardInterrupt: 

In [ ]:
n_acc_samples / n_trial

In [ ]:
acccepted_x = np.array(acccepted_x)
cand_x = np.array(cand_x)

In [ ]:
acccepted_x.shape


In [ ]:
cand_x.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
xtrue = xtrue.squeeze()

In [ ]:
cand_x.shape

In [ ]:
for i in range(cand_x.shape[0]):
    plt.plot(cand_x[i,:,0], color = 'gray')
    
for i in range(acccepted_x.shape[0]):
    plt.plot(acccepted_x[i,:,0], color = 'blue')

plt.plot(xtrue, color = 'red')

In [ ]:
pd.DataFrame(cand_x[:,:,0].T).plot()

In [ ]:
acccepted_x[:,:,0].shape

In [ ]:
plt.plot(acccepted_x[0,:,0])
plt.plot(acccepted_x[1,:,0])

In [ ]:
pacc=exp(logZ-logZbar)
            u=rand(1);
            if u<pacc
                xacc(p,:)=xcand;
                flag=0;
            end